In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_schema = "order_id long , order_date date, customer_id long,order_status string"

In [3]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [4]:
spark.sql("create database itv006277_cachingdemo_db")

""


Writing the dataframe into table

In [ ]:
orders_df.write.format("csv").saveAsTable("itv006277_cachingdemo_db.itv006277_orders1")

In [ ]:
spark.sql("describe extended itv006277_cachingdemo_db.itv006277_orders1").show()

In [ ]:
spark.sql("select count(*) from itv006277_cachingdemo_db.itv006277_orders1").show()

Taking Cache 

In [ ]:
spark.sql("cache table itv006277_cachingdemo_db.itv006277_orders1")

In [ ]:
spark.sql("select count(*) from itv006277_cachingdemo_db.itv006277_orders1").show()

Here Spark will create the multiple job internally if you fire the below query

In [ ]:
spark.sql("select distinct(order_status) from itv006277_cachingdemo_db.itv006277_orders1").show()

Here Spark will create single job for the below query

In [ ]:
spark.sql("select count(distinct(order_status)) from itv006277_cachingdemo_db.itv006277_orders1").show()

Taking Un_Cache 

In [ ]:
spark.sql("uncache table itv006277_cachingdemo_db.itv006277_orders1")

By default saprk sql is eager 

changing from eager to lazy

In [ ]:
spark.sql("cache lazy table itv006277_cachingdemo_db.itv006277_orders1")

Now It will hit cache 

check it under the storage

In [ ]:
spark.sql("select count(distinct(order_status)) from itv006277_cachingdemo_db.itv006277_orders1").show()

This wil hit cache 

In [ ]:
spark.sql("select order_status, count(*) from itv006277_cachingdemo_db.itv006277_orders1 group by order_status").show()

As we know that we have created cache and If i inssert or any changes in data (cache)

now cache data is not correct u

In [ ]:
spark.sql("insert into itv006277_cachingdemo_db.itv006277_orders1 values(111111, '2023-05-29', 222222, 'BOOKED')")

The Below query will not hit cache and take time  and please check it 

we need to refresh tha cache

In [ ]:
spark.sql("select count(distinct(order_status)) from itv006277_cachingdemo_db.itv006277_orders1").show()

If i run the below query and this will hit cache because the cache will correct

In [ ]:
spark.sql("select count(distinct(order_status)) from itv006277_cachingdemo_db.itv006277_orders1").show()

How to uncache everything ?

In [ ]:
spark.sql("clear cache")

How to uncache specfic table ?

In [ ]:
spark.sql("uncache table itv006277_cachingdemo_db.itv006277_orders1")